<a href="https://colab.research.google.com/github/semthedev/ml-course-2025/blob/main/seminars/03_pandas_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jupyter_black

In [ ]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


# Pandas

http://pandas.pydata.org/

Exploratory data analysis, предобработка данных, predictive modeling (в малой степени).
Проекции, слияние, фильтрация, группировка, агрегация, одним словом - **работа с таблицами.**

**Есть смысл читать документацию и думать о наибольшей эффективности выполнения тех или иных операций**.
Так, по-прежнему, если вы пишете цикл, вы что-то делаете не так; векторные операции for the win! И особенно это важно, если ваш код будет затем адаптироваться для [py]Spark. Благодаря DataFrame API Apache Spark, это делается просто.

Дополнительно с pandas можно ознакомиться тут:
1. [Более короткий](https://github.com/jvns/pandas-cookbook)
2. [Более обстоятельный](https://github.com/guipsamora/pandas_exercises/tree/master).

Если будете решать вторую — обязательно пройдите первые пару глав из первой, так как во второй тема индексаций пропущена полностью :)

## Data structures & types

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data=[[1, 2, 3], [4, 5, 6]],
    columns=["A", "B", "C"],
    index=["X", "Y"],
)
df.head()

,A,B,C
X,1,2,3
Y,4,5,6


In [ ]:
df.tail(1)

,A,B,C
Y,4,5,6


In [ ]:
df.sample(10, replace=True, random_state=123)

,A,B,C
X,1,2,3
Y,4,5,6
X,1,2,3
X,1,2,3
X,1,2,3
X,1,2,3
X,1,2,3
Y,4,5,6
Y,4,5,6
X,1,2,3


In [ ]:
data = {"X": [1, 2, 3], "Y": [4, 5, 6]}
df = pd.DataFrame.from_dict(data)

df.head()

,X,Y
0,1,4
1,2,5
2,3,6


In [ ]:
data = {"X": [1, 2, 3], "Y": [4, 5, 6]}
df = pd.DataFrame.from_dict(data, orient="index", columns=["A", "B", "C"])

df.head()

,A,B,C
X,1,2,3
Y,4,5,6


In [ ]:
df["E"] = 0
df

,A,B,C,D,E
X,1,2,3,a,0
Y,4,5,6,b,0


In [ ]:
type(df["A"])

pandas.core.series.Series

In [ ]:
df[["A"]].columns

Index(['A'], dtype='object')

In [ ]:
type(df[["A"]])

pandas.core.frame.DataFrame

In [ ]:
df.A

,A
X,1
Y,4


In [ ]:
df.shape

(2, 5)

In [ ]:
# series -- "колонка" одного типа; можно относиться как к словарю из ИНДЕКСОВ в ЗНАЧЕНИЯ
s = pd.Series([1, 3, 5, np.nan, 6, 8], name="Some series")

print("Values types: ", s.dtype)
print("Series shape: ", s.shape)
print("Series indeix:", s.index)

s.values
s[2:40:2]
s.index
# s

Values types:  float64
Series shape:  (6,)
Series indeix: RangeIndex(start=0, stop=6, step=1)


RangeIndex(start=0, stop=6, step=1)

In [ ]:
s = pd.Series(
    [1, 3, 5, np.nan, 6, 8, 10],
    name="Some series",
    index=["q", "w", "e", "e", "r", "t", "y"],
)
s
print("Index:", s.index)
print(s[1:5])

print(s["w":"e"])  # обратите внимание!
# 1) порядок -- лексикографический?
# 2) включён ли последний индекс?
# 3) а если сделать s["w":"z"]?

print(s["e":"q"])  # обратите внимание!

Index: Index(['q', 'w', 'e', 'e', 'r', 't', 'y'], dtype='object')
w    3.0
e    5.0
e    NaN
r    6.0
Name: Some series, dtype: float64
w    3.0
e    5.0
e    NaN
Name: Some series, dtype: float64
Series([], Name: Some series, dtype: float64)


In [ ]:
# А так ли все работает целочисленными индексами?
t = pd.Series(
    [1, 3, 5, np.nan, 6, 8, 10],
    name="Some series",
    index=[1, 2, 3, 4, 5, 6, 7],
)

# разница, на самом деле, заключена вот здесь, но об этом чуть позже
t.iloc[1:5], t.loc[1:5], s.iloc[1:5],  # s.loc[1:5]

(2    3.0
 3    5.0
 4    NaN
 5    6.0
 Name: Some series, dtype: float64,
 1    1.0
 2    3.0
 3    5.0
 4    NaN
 5    6.0
 Name: Some series, dtype: float64,
 w    3.0
 e    5.0
 e    NaN
 r    6.0
 Name: Some series, dtype: float64)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, X to Y
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       2 non-null      int64 
 1   B       2 non-null      int64 
 2   C       2 non-null      int64 
 3   D       2 non-null      object
 4   E       2 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 204.0+ bytes


In [ ]:
df.describe()

,A,B,C,E
count,2.00000,2.00000,2.00000,2.0
mean,2.50000,3.50000,4.50000,0.0
std,2.12132,2.12132,2.12132,0.0
min,1.00000,2.00000,3.00000,0.0
25%,1.75000,2.75000,3.75000,0.0
50%,2.50000,3.50000,4.50000,0.0
75%,3.25000,4.25000,5.25000,0.0
max,4.00000,5.00000,6.00000,0.0


In [ ]:
df.dtypes

,0
A,int64
B,int64
C,int64
D,object
E,int64


В пандас есть бесчисленное количество методов для работы с табличными данными, и постоянно добавляется новое, поэтому есть смысл почитать к нему документацию, а не только чужой код. Может быть, так вам удастся избежать переизобретения велосипеда.

Тем, кто работает с реальными временными рядами, могут пригодиться вот такие штучки:

In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
dates = pd.date_range("20130101", periods=6)  # default frequency measure is 1 day

print(dates)

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')


Потихоньку переходим к главному объекту пандаса: датафреймам.


In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
df = pd.DataFrame(
    data=np.random.randn(6, 4),
    index=dates,
    columns=list("ABCD"),
)

df

,A,B,C,D
2013-01-01,0.145834,0.632482,-1.036795,1.219148
2013-01-02,1.124432,0.743660,-1.256744,0.398740
2013-01-03,0.448825,0.175004,0.034020,0.733624
2013-01-04,0.776796,-0.017745,1.103409,-1.159509
2013-01-05,-0.411061,-0.856197,-0.417581,-1.195131
2013-01-06,0.646537,-1.154022,-0.113079,0.650285


Ещё один способ задать датафрейм: ключи -- имена колонок, значения -- сами колонки

In [ ]:
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20130102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)
df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [ ]:
df2.dtypes

,0
A,float64
B,datetime64[s]
C,float32
D,int32
E,category
F,object


## Accessing data
Как посмотреть на набор данных и что-то о нём понять

In [ ]:
df.head(3)  # , df.tail(2)

,A,B,C,D
2013-01-01,0.145834,0.632482,-1.036795,1.219148
2013-01-02,1.124432,0.743660,-1.256744,0.398740
2013-01-03,0.448825,0.175004,0.034020,0.733624


In [ ]:
print(df.index)
print(df.columns)
print(df.values)

df2.values

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')
Index(['A', 'B', 'C', 'D'], dtype='object')
[[ 0.14583369  0.63248158 -1.03679506  1.21914827]
 [ 1.12443246  0.74366018 -1.25674372  0.39874021]
 [ 0.44882517  0.17500425  0.03401986  0.73362437]
 [ 0.77679616 -0.01774524  1.1034085  -1.15950875]
 [-0.4110607  -0.85619702 -0.41758115 -1.19513058]
 [ 0.64653683 -1.15402227 -0.11307906  0.65028494]]


array([[1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'test', 'foo'],
       [1.0, Timestamp('2013-01-02 00:00:00'), 1.0, 3, 'train', 'foo']],
      dtype=object)

In [ ]:
# стандартные статистики по каждой колонке
df.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,0.455227,-0.079470,-0.281128,0.107860
std,0.535549,0.776009,0.846874,1.030474
min,-0.411061,-1.154022,-1.256744,-1.195131
25%,0.221582,-0.646584,-0.881992,-0.769947
50%,0.547681,0.078630,-0.265330,0.524513
75%,0.744231,0.518112,-0.002755,0.712790
max,1.124432,0.743660,1.103409,1.219148


In [ ]:
df.T

,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06
A,0.145834,1.124432,0.448825,0.776796,-0.411061,0.646537
B,0.632482,0.743660,0.175004,-0.017745,-0.856197,-1.154022
C,-1.036795,-1.256744,0.034020,1.103409,-0.417581,-0.113079
D,1.219148,0.398740,0.733624,-1.159509,-1.195131,0.650285


In [ ]:
df.T.index, df.index

(Index(['A', 'B', 'C', 'D'], dtype='object'),
 DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
                '2013-01-05', '2013-01-06'],
               dtype='datetime64[ns]', freq='D'))

In [ ]:
df.sort_index(axis=1, ascending=False), df

(                   D         C         B         A
 2013-01-01  1.219148 -1.036795  0.632482  0.145834
 2013-01-02  0.398740 -1.256744  0.743660  1.124432
 2013-01-03  0.733624  0.034020  0.175004  0.448825
 2013-01-04 -1.159509  1.103409 -0.017745  0.776796
 2013-01-05 -1.195131 -0.417581 -0.856197 -0.411061
 2013-01-06  0.650285 -0.113079 -1.154022  0.646537,
                    A         B         C         D
 2013-01-01  0.145834  0.632482 -1.036795  1.219148
 2013-01-02  1.124432  0.743660 -1.256744  0.398740
 2013-01-03  0.448825  0.175004  0.034020  0.733624
 2013-01-04  0.776796 -0.017745  1.103409 -1.159509
 2013-01-05 -0.411061 -0.856197 -0.417581 -1.195131
 2013-01-06  0.646537 -1.154022 -0.113079  0.650285)

In [ ]:
df.sort_values(by="B")

,A,B,C,D
2013-01-06,0.646537,-1.154022,-0.113079,0.650285
2013-01-05,-0.411061,-0.856197,-0.417581,-1.195131
2013-01-04,0.776796,-0.017745,1.103409,-1.159509
2013-01-03,0.448825,0.175004,0.034020,0.733624
2013-01-01,0.145834,0.632482,-1.036795,1.219148
2013-01-02,1.124432,0.743660,-1.256744,0.398740


## Выборка данных
#### NB!
While standard Python / Numpy expressions for selecting and setting are intuitive and come in handy for interactive work, for production code, we recommend the optimized pandas data access methods, .at, .iat, .loc, .iloc and .ix.

In [ ]:
df["A"]

,A
2013-01-01,0.145834
2013-01-02,1.124432
2013-01-03,0.448825
2013-01-04,0.776796
2013-01-05,-0.411061
2013-01-06,0.646537


In [ ]:
df[["A", "B"]]

,A,B
2013-01-01,0.145834,0.632482
2013-01-02,1.124432,0.743660
2013-01-03,0.448825,0.175004
2013-01-04,0.776796,-0.017745
2013-01-05,-0.411061,-0.856197
2013-01-06,0.646537,-1.154022


In [ ]:
df.A

,A
2013-01-01,0.145834
2013-01-02,1.124432
2013-01-03,0.448825
2013-01-04,0.776796
2013-01-05,-0.411061
2013-01-06,0.646537


In [ ]:
df[:]  # по строкам

,A,B,C,D
2013-01-01,0.145834,0.632482,-1.036795,1.219148
2013-01-02,1.124432,0.743660,-1.256744,0.398740
2013-01-03,0.448825,0.175004,0.034020,0.733624
2013-01-04,0.776796,-0.017745,1.103409,-1.159509
2013-01-05,-0.411061,-0.856197,-0.417581,-1.195131
2013-01-06,0.646537,-1.154022,-0.113079,0.650285


## Выборка по значениям индексов

In [ ]:
df["20130102":"20130104"]

,A,B,C,D
2013-01-02,1.124432,0.743660,-1.256744,0.398740
2013-01-03,0.448825,0.175004,0.034020,0.733624
2013-01-04,0.776796,-0.017745,1.103409,-1.159509


In [ ]:
# первый индекс
print("First index: " + str(dates[0]))

print(df)

# выбираем всё по этому индексу
print(df.loc[dates[0]])

First index: 2013-01-01 00:00:00
                   A         B         C         D
2013-01-01  0.145834  0.632482 -1.036795  1.219148
2013-01-02  1.124432  0.743660 -1.256744  0.398740
2013-01-03  0.448825  0.175004  0.034020  0.733624
2013-01-04  0.776796 -0.017745  1.103409 -1.159509
2013-01-05 -0.411061 -0.856197 -0.417581 -1.195131
2013-01-06  0.646537 -1.154022 -0.113079  0.650285
A    0.145834
B    0.632482
C   -1.036795
D    1.219148
Name: 2013-01-01 00:00:00, dtype: float64


## Выборка по порядковым номерам индексов

In [ ]:
df.iloc[3:5]

,A,B,C,D
2013-01-04,0.776796,-0.017745,1.103409,-1.159509
2013-01-05,-0.411061,-0.856197,-0.417581,-1.195131


In [ ]:
# доступ до отдельного значения
print(df.iat[1, 1])
print(df.iloc[1, 1])  # то же

0.7436601771319268
0.7436601771319268


In [ ]:
df.A > 0

,A
2013-01-01,True
2013-01-02,True
2013-01-03,True
2013-01-04,True
2013-01-05,False
2013-01-06,True


In [ ]:
# выборка по значениям в колонках [ничего не напоминает?]
df[df.A > 0]

,A,B,C,D
2013-01-01,0.145834,0.632482,-1.036795,1.219148
2013-01-02,1.124432,0.743660,-1.256744,0.398740
2013-01-03,0.448825,0.175004,0.034020,0.733624
2013-01-04,0.776796,-0.017745,1.103409,-1.159509
2013-01-06,0.646537,-1.154022,-0.113079,0.650285


In [ ]:
# фильтрация
df2 = df.copy()
df2["E"] = ["one", "one", "two", "three", "four", "three"]

# то же, что filter по вхождению
# df2[]
df2[df2["E"].isin(["one"])]
df2["E"].isin(["hello", "woow", "three", "one"])

,E
2013-01-01,True
2013-01-02,True
2013-01-03,False
2013-01-04,True
2013-01-05,False
2013-01-06,True


## Обновление

In [ ]:
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=pd.date_range("20130102", periods=6))
df["F"] = s1
df

,A,B,C,D,F
2013-01-01,0.145834,0.632482,-1.036795,1.219148,NaN
2013-01-02,1.124432,0.743660,-1.256744,0.398740,1.0
2013-01-03,0.448825,0.175004,0.034020,0.733624,2.0
2013-01-04,0.776796,-0.017745,1.103409,-1.159509,3.0
2013-01-05,-0.411061,-0.856197,-0.417581,-1.195131,4.0
2013-01-06,0.646537,-1.154022,-0.113079,0.650285,5.0


In [ ]:
# выборка по индексам
df.at["2013-01-04", "A"] = 99.0
df.iat[0, 1] = 999999
df.loc[:, "D"] = np.array([5] * len(df))
df

,A,B,C,D,F
2013-01-01,0.145834,999999.000000,-1.036795,5.0,NaN
2013-01-02,1.124432,0.743660,-1.256744,5.0,1.0
2013-01-03,0.448825,0.175004,0.034020,5.0,2.0
2013-01-04,99.000000,-0.017745,1.103409,5.0,3.0
2013-01-05,-0.411061,-0.856197,-0.417581,5.0,4.0
2013-01-06,0.646537,-1.154022,-0.113079,5.0,5.0


In [ ]:
df < -0.5

,A,B,C,D,F
2013-01-01,False,False,False,False,False
2013-01-02,False,False,False,False,False
2013-01-03,False,False,False,False,False
2013-01-04,False,False,False,False,False
2013-01-05,False,False,False,False,False
2013-01-06,False,False,False,False,False


In [ ]:
# выборка по условию
df[df < -0.5] = 0 * df
df
# df > 0

,A,B,C,D,F
2013-01-01,0.145834,999999.000000,-0.000000,5.0,NaN
2013-01-02,1.124432,0.743660,-0.000000,5.0,1.0
2013-01-03,0.448825,0.175004,0.034020,5.0,2.0
2013-01-04,99.000000,-0.017745,1.103409,5.0,3.0
2013-01-05,-0.411061,-0.000000,-0.417581,5.0,4.0
2013-01-06,0.646537,-0.000000,-0.113079,5.0,5.0


In [ ]:
# df[uslovie(df.ix)] = func(df)

In [ ]:
df["cat_variable"] = ["Barsik", "Barsik", "Marusya", "Barsik", "Solomon", "Marusya"]

In [ ]:
df

,A,B,C,D,F,cat_variable
2013-01-01,-0.000000,999999.000000,-0.174799,5.0,NaN,Barsik
2013-01-02,-0.000000,-0.000000,-0.000000,5.0,1.0,Barsik
2013-01-03,0.502622,0.104237,0.277107,5.0,2.0,Marusya
2013-01-04,99.000000,1.503947,-0.066953,5.0,3.0,Barsik
2013-01-05,0.587008,-0.000000,1.425868,5.0,4.0,Solomon
2013-01-06,1.047034,0.977861,-0.000000,5.0,5.0,Marusya


In [ ]:
cats_df = pd.get_dummies(df["cat_variable"], prefix="cat")
cats_df

,cat_Barsik,cat_Marusya,cat_Solomon
2013-01-01,True,False,False
2013-01-02,True,False,False
2013-01-03,False,True,False
2013-01-04,True,False,False
2013-01-05,False,False,True
2013-01-06,False,True,False


In [ ]:
vals = pd.concat([df.drop(["F", "A", "cat_variable"], axis=1), cats_df], axis=1)
vals

,B,C,D,cat_Barsik,cat_Marusya,cat_Solomon
2013-01-01,999999.000000,-0.174799,5.0,True,False,False
2013-01-02,-0.000000,-0.000000,5.0,True,False,False
2013-01-03,0.104237,0.277107,5.0,False,True,False
2013-01-04,1.503947,-0.066953,5.0,True,False,False
2013-01-05,-0.000000,1.425868,5.0,False,False,True
2013-01-06,0.977861,-0.000000,5.0,False,True,False


In [ ]:
df.drop(["F", "A", "cat_variable"], axis=1)

,B,C,D
2013-01-01,999999.000000,-0.174799,5.0
2013-01-02,-0.000000,-0.000000,5.0
2013-01-03,0.104237,0.277107,5.0
2013-01-04,1.503947,-0.066953,5.0
2013-01-05,-0.000000,1.425868,5.0
2013-01-06,0.977861,-0.000000,5.0


На самом деле, в pandas есть довольно продвинутый sql-like синтаксис, но нам вряд ли он понадобится.

In [ ]:
# sample data
orders_data = {
    "customer_id": [1, 2, 3, 1, 2, 3, 4],
    "order_amount": [100, 200, 150, 300, 120, 50, 500],
}
orders = pd.DataFrame(orders_data)

customers_data = {
    "customer_id": [1, 2, 3, 4],
    "customer_name": ["Alice", "Bob", "Charlie", "David"],
    "city": ["New York", "Los Angeles", "New York", "Chicago"],
}
customers = pd.DataFrame(customers_data)

In [ ]:
# merge orders with customers on customer_id
merged_df = pd.merge(orders, customers, on="customer_id")
merged_df

,customer_id,order_amount,customer_name,city
0,1,100,Alice,New York
1,2,200,Bob,Los Angeles
2,3,150,Charlie,New York
3,1,300,Alice,New York
4,2,120,Bob,Los Angeles
5,3,50,Charlie,New York
6,4,500,David,Chicago


In [ ]:
# Group by city and aggregate the total revenue per city
city_revenue = (
    merged_df.groupby("city")["order_amount"].sum().reset_index(name="total_revenue")
)
city_revenue

,city,total_revenue
0,Chicago,500
1,Los Angeles,320
2,New York,600


In [ ]:
# apply a custom function to find the top customer (by total order amount) in each city
def top_customer_in_city(df):
    top_customer = df.groupby("customer_name")["order_amount"].sum().idxmax()
    return top_customer


# group by city and apply the custom function to find the top customer in each city
top_customers = (
    merged_df.groupby("city")
    .apply(top_customer_in_city)
    .reset_index(name="top_customer")
)
top_customers

/tmp/ipython-input-458105633.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(top_customer_in_city)


,city,top_customer
0,Chicago,David
1,Los Angeles,Bob
2,New York,Alice


---
## Упражнения
*Не являются домашкой

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("estonia-passenger-list.csv")
## если видеть не можете "Титаник":
## https://www.kaggle.com/christianlillelund/passenger-list-for-the-estonia-ferry-disaster
## Задание 0
## поисследуйте распределения значений фич --
# print(df["Country"].value_counts().head(), "\n")
# df.describe()
## Что делает эта цепочка вызовов? Какой объект после каждого? Что показывает?
# print(df.isna().sum())

In [ ]:
df.head(2)

,PassengerId,Country,Firstname,Lastname,Sex,Age,Category,Survived
0,1,Sweden,ARVID KALLE,AADLI,M,62,P,0
1,2,Estonia,LEA,AALISTE,F,22,C,0


In [ ]:
# pd.get_dummies(df["Category"])
# pd.get_dummies(df["Category"], prefix="Category")

In [ ]:
# Задание 2
# Написать код, заполняющий пропуски в "численных" колонках
# 2a -- значением (-1)
# 2b -- средним значением по колонке

In [ ]:
# Задание 3
# Понять, какие фичи -- номинальные (категориальные) -- без осмысленного порядка над ними.
# Применить к ним пандасовский dummy_encoding

In [ ]:
# Задание 4
# Нормализуйте средствами pandas (нельзя использовать sklearn) отдельно
# каждую колонку -- с возрастом и со стоимостью билета
# 5a. min-max scaling
# 5b. вычесть среднее и разделить на стандартное отклонение

In [ ]:
# Задание 5
# Постройте
# train.pivot_table('PassengerId', 'Pclass', 'Survived', 'count').plot(kind='bar', stacked=True)
# Погуглите, что это? О чём нам говорит этот график?

In [ ]:
# Задание 6
# Для каждой фичи, кажущейся вам полезной, постройте гистограмму с помощью pandas-hist

**P.S.** Задачи, похожие на эти упражнения, в мире машинного обучения и статистики называются Exploratory Data Analysis (EDA). Наш курс смещен в сторону теории обучения, и заниматься подобным мы больше не будем. Тем, кому надо, советую пройти курсы по feature engineering, data visualization, & data cleaning в Kaggle Learn.


## Задача 4. Split.

Реализуйте разбиение датасета на train, test и val при помощи pandas и без использования циклов на Python. Разбиение должно быть стратифицировано по колонкам, данные должны быть перемешаны. Подробно объясните и/или прокомментируйте, почему ваш код делает то, что нужно.

In [ ]:
def split_stratified(df, stratify_clumns, train_frac=0.6, val_frac=0.2): ...


train, val, test = split_stratified(df, ["Category", "Survived"])

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
import numpy as np
import pandas as pd

def split_stratified(df, stratify_cols, train_frac=0.6, val_frac=0.2, seed=42):
    g = df.groupby(stratify_cols, group_keys=False)

    train  = g.sample(frac=train_frac, random_state=seed)
    remain = df.drop(train.index)

    val = remain.groupby(stratify_cols, group_keys=False).sample(
        frac=val_frac / (1 - train_frac), random_state=seed + 1
    )
    test = remain.drop(val.index)

    # финальное перемешивание (по желанию)
    train = train.sample(frac=1.0, random_state=seed    ).reset_index(drop=True)
    val   = val.sample(  frac=1.0, random_state=seed + 1).reset_index(drop=True)
    test  = test.sample( frac=1.0, random_state=seed + 2).reset_index(drop=True)
    return train, val, test


In [ ]:
import numpy as np
import pandas as pd

def split_stratified(df, stratify_cols, train_frac=0.6, val_frac=0.2, seed=42):
    """
    Делит DataFrame на train/val/test БЕЗ циклов и с сохранением долей по стратам.

    Идея простая:
    1) внутри каждой страты (группы по `stratify_cols`) берём случайную выборку
       размера ~ train_frac -> это и есть train;
    2) из оставшегося (df \ train) снова по стратам берём кусок под val
       с долей, пересчитанной относительно остатка: val_frac/(1-train_frac);
    3) всё, что не попало в train/val, идёт в test;
    4) в конце перемешиваем части, чтобы убрать порядок исходного df.

    Важные свойства:
    - стратификация: выборка делается для КАЖДОЙ группы отдельно (groupby.sample),
      поэтому пропорции классов/категорий сохраняются;
    - воспроизводимость: фиксируем случайное зерно (seed);
    - части непересекаются: на втором шаге выбираем из `remain`, а не из исходного df.
    """
    # группируем по стратам; group_keys=False — чтобы не добавлять уровень
    # индекса с названиями групп при конкатенации результатов
    g = df.groupby(stratify_cols, group_keys=False)

    # 1) train: из каждой страты берём train_frac случайных строк
    train = g.sample(frac=train_frac, random_state=seed)

    # 2) остаток после train
    remain = df.drop(train.index)

    # 3) val: из остатка снова по стратам; долю пересчитываем от остатка
    val = remain.groupby(stratify_cols, group_keys=False).sample(
        frac=val_frac / (1 - train_frac), random_state=seed + 1
    )

    # 4) test — просто всё, что не вошло в val из остатка
    test = remain.drop(val.index)

    # 5) финальное перемешивание (опционально), чтобы разорвать исходный порядок
    train = train.sample(frac=1.0, random_state=seed    ).reset_index(drop=True)
    val   = val.sample(  frac=1.0, random_state=seed + 1).reset_index(drop=True)
    test  = test.sample( frac=1.0, random_state=seed + 2).reset_index(drop=True)

    return train, val, test
